<a name="top"> <h1>01. ML Models</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>

<p>&nbsp;</p>

<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[1. Librerias utilizadas y funciones](#librerias) 

[2. Introducción ](#introduccion) 

   - Objetivo de negocio.

[3. Yelp Dataset ](#yelp) 

   - Información del dataset
   - Características del dataset


[4. Transformación del formato de ficheros](#transformacion) 


[5. Transformación de datos](#datos)

   - Business
       - Carga del fichero
       - Transformación de los datos
       - Exportación de ficheros procesados

<hr style="border:1px solid gray">

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para el preprocesamiento:

In [1]:
# Import libraries.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from lazypredict.Supervised import LazyClassifier
import numpy as np
import string

# <a name="lectura"> 2. Lectura del dataframe <a>


In [2]:
#Import parquet file.

reviews = pd.read_parquet('../data/processed/reviews.parquet')

# Show the head of the dataframe.

reviews.head()

,reviewText,overall
0,amazon shopping amazoncom gift cards christmas...,1
1,got gift card friend best site much choose gre...,5
2,arent going save trees people complaining pape...,5
3,always get someone something amazon safety net...,5
4,take 50 dollars good money limitations turn am...,1


In [3]:
X = reviews.reviewText
y = reviews.overall

In [4]:
# Split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [6]:

tfidf = TfidfVectorizer(stop_words="english",max_df=0.99,min_df=0.01)
svc = SVC()
model = make_pipeline(tfidf, svc)
#Training
model.fit(x_train, y_train)

#Prediccion sobre test
preds= model.predict(x_test )


In [7]:
np.mean(preds==y_test)

0.8723122388668093

In [8]:
# import multionomial naive bayes model

from sklearn.naive_bayes import MultinomialNB

# Entrenar el modelo
clf = MultinomialNB()

model = make_pipeline(tfidf, clf)
#Training
model.fit(x_train, y_train)

#Prediccion sobre test
preds= model.predict(x_test )


In [9]:
np.mean(preds==y_test)

0.8631407316824621

In [10]:
# Entrene con modelo xgboost

from xgboost import XGBClassifier

# Entrenar el modelo

xgb = XGBClassifier()

model = make_pipeline(tfidf, xgb)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

np.mean(preds==y_test)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4], got [1 2 3 4 5]

In [11]:
# Entrene con modelo random forest

from sklearn.ensemble import RandomForestClassifier

# Entrenar el modelo

rf = RandomForestClassifier()

model = make_pipeline(tfidf, rf)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

np.mean(preds==y_test)

0.8662318692890384

In [12]:
# Train with logistic regression

from sklearn.linear_model import LogisticRegression

# Entrenar el modelo

lr = LogisticRegression()

model = make_pipeline(tfidf, lr)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

np.mean(preds==y_test)



0.8713271510581202

In [ ]:
# Train with a SVM

from sklearn.svm import SVC

# Entrenar el modelo

svc = SVC()

